In [58]:
import pandas as pd
import numpy as np

df = pd.read_csv('full_merged_with_clusters_compared.csv')
clusters = sorted(df['cluster'].dropna().unique().tolist())

print(f'클러스터 목록 ({len(clusters)}개): {clusters}')
print('가용 컬럼 목록:')
for column in df.columns:
    print(f' - {column}')


클러스터 목록 (4개): [0, 1, 2, 3]
가용 컬럼 목록:
 - 가맹점_구분번호
 - 기준년월
 - 운영개월수_구간
 - 매출금액_구간
 - 매출건수_구간
 - 유니크고객수_구간
 - 객단가_구간
 - 취소율_구간
 - 배달매출_비율
 - 동일업종_매출금액비율
 - 동일업종_매출건수비율
 - 동일업종_내매출순위비율
 - 동일상권_내매출순위비율
 - 동일업종_내해지가맹점비중
 - 동일상권_내해지가맹점비중
 - 업종
 - 상권
 - 개설일
 - 남성_20대이하_비중
 - 남성_30대_비중
 - 남성_40대_비중
 - 남성_50대_비중
 - 남성_60대이상_비중
 - 여성_20대이하_비중
 - 여성_30대_비중
 - 여성_40대_비중
 - 여성_50대_비중
 - 여성_60대이상_비중
 - 재방문_고객비중
 - 신규_고객비중
 - 거주고객_비율
 - 직장인고객_비율
 - 유동인구고객_비율
 - male_total_ratio
 - female_total_ratio
 - cluster
 - label


In [61]:
# 확장된 변수 목록 정의
vars_extended = [
    '운영개월수_구간',
    '유니크고객수_구간',
    '객단가_구간',
    '취소율_구간',
    '배달매출_비율',
    '동일상권_내매출순위비율',
    '남성_20대이하_비중',
    '남성_30대_비중',
    '남성_40대_비중',
    '남성_50대_비중',
    '남성_60대이상_비중',
    '여성_20대이하_비중',
    '여성_30대_비중',
    '여성_40대_비중',
    '여성_50대_비중',
    '여성_60대이상_비중',
    '재방문_고객비중',
    '신규_고객비중',
    '거주고객_비율',
    '직장인고객_비율',
    '유동인구고객_비율',
]

# 실제 존재하는 변수만 사용
vars_extended_use = [v for v in vars_extended if v in df.columns]
print(f'확장 변수 목록 ({len(vars_extended_use)}개):')
for i, v in enumerate(vars_extended_use, 1):
    print(f'  {i}. {v}')

# 없는 변수 확인
missing_vars = [v for v in vars_extended if v not in df.columns]
if missing_vars:
    print(f'\n⚠️ 데이터에 없는 변수 ({len(missing_vars)}개):')
    for v in missing_vars:
        print(f'  - {v}')

확장 변수 목록 (21개):
  1. 운영개월수_구간
  2. 유니크고객수_구간
  3. 객단가_구간
  4. 취소율_구간
  5. 배달매출_비율
  6. 동일상권_내매출순위비율
  7. 남성_20대이하_비중
  8. 남성_30대_비중
  9. 남성_40대_비중
  10. 남성_50대_비중
  11. 남성_60대이상_비중
  12. 여성_20대이하_비중
  13. 여성_30대_비중
  14. 여성_40대_비중
  15. 여성_50대_비중
  16. 여성_60대이상_비중
  17. 재방문_고객비중
  18. 신규_고객비중
  19. 거주고객_비율
  20. 직장인고객_비율
  21. 유동인구고객_비율


## BOSS 알고리즘 적용

BOSS (Best Order Score Search)는 변수들의 인과 순서를 찾는 최적화 기반 알고리즘입니다.
- **특징**: BIC 스코어를 최대화하는 변수 순서를 찾음
- **장점**: 대규모 변수에도 확장 가능, 계산 효율적
- **출력**: 변수들의 인과 순서와 각 변수의 부모 노드

In [62]:
# BOSS 알고리즘 관련 모듈 import
import sys
sys.path.append('/Users/kim-bogeun/Downloads/빅콘/boss-main')

from boss import boss
from gst import GST
from scores import BIC

print('✅ BOSS 모듈 import 완료')

✅ BOSS 모듈 import 완료


In [78]:
# BOSS 알고리즘 실행 함수

def run_boss_on_df(df_sub, cols, penalty_discount=2.0):
    """
    BOSS 알고리즘을 사용하여 인과 순서를 발견
    
    Parameters:
    -----------
    df_sub: DataFrame - 분석할 데이터
    cols: list - 변수 이름 리스트
    penalty_discount: float - penalty discount (BIC 패널티 조정)
    
    Returns:
    --------
    dict - 인과 순서, 부모 노드, 변수 수, 표본 수
    """
    # 숫자형 변환 및 결측 제거
    X = df_sub[list(cols)].copy()

    for column in X.columns:
        if pd.api.types.is_numeric_dtype(X[column]):
            X[column] = pd.to_numeric(X[column], errors='coerce')
        else:
            X[column] = X[column].astype('category').cat.codes.replace(-1, np.nan)

    X = X.dropna()
    if len(X) < 20:
        print('표본이 적어 BOSS 생략')
        return None
    
    n, p = X.shape
    X_np = X.values
    
    # BIC 스코어 객체 생성
    score = BIC(X_np, pd=penalty_discount)
    
    # 초기 순서 (0, 1, 2, ..., p-1)
    order = [v for v in range(p)]
    
    # 각 변수에 대한 GST 생성
    gsts = [GST(v, score) for v in order]
    
    # BOSS 알고리즘 실행 (순서 최적화)
    from numpy.random import default_rng
    rng = default_rng(135)  # 재현성을 위한 seed
    rng.shuffle(order)
    boss(order, gsts)
    
    # 각 변수의 부모 노드 추적
    parents = {v: [] for v in order}
    for i, v in enumerate(order):
        gsts[v].trace(order[:i], parents[v])
    
    # 변수 이름으로 변환
    order_names = [cols[v] for v in order]
    parents_names = {cols[v]: [cols[p] for p in parents[v]] for v in parents}
    
    return {
        'n': n,
        'p': p,
        'order': order,
        'order_names': order_names,
        'parents': parents,
        'parents_names': parents_names,
        'score': score
    }

## BOSS 인과 분석

이제 더 많은 변수를 포함하여 인과 관계를 분석합니다.

In [79]:
# 확장된 변수셋으로 클러스터별 BOSS 분석 실행
print('=' * 80)
print('확장된 변수셋 BOSS 인과 분석')
print('=' * 80)
print()

boss_extended_results = {}

for cl in clusters:
    sub = df[df['cluster'] == cl]
    print(f"\n{'='*80}")
    print(f"Cluster {cl} 분석")
    print(f"{'='*80}")
    
    res = run_boss_on_df(sub, vars_extended, penalty_discount=2.0)
    if res is None:
        print('분석 불가 (표본 부족)\n')
        continue
    
    boss_extended_results[cl] = res
    print(f"\n📊 기본 정보:")
    print(f"  - 표본 수: {res['n']:,}개")
    print(f"  - 변수 수: {res['p']}개")
    print(f"  - 발견된 인과 관계: {sum(len(res['parents_names'][v]) for v in res['parents_names'])}개")
    
    # 인과 순서
    print(f"\n🔄 인과 순서 (왼쪽 → 오른쪽):")
    print(f"  {' → '.join(res['order_names'])}")
    
    # 원인 변수 (부모가 없는 변수들)
    root_causes = [v for v in res['order_names'] if not res['parents_names'][v]]
    print(f"\n🌟 원인 변수 (Root Causes) - {len(root_causes)}개:")
    if root_causes:
        for var in root_causes:
            print(f"  ✓ {var}")
    else:
        print("  (없음)")
    
    # 인과 관계 상세 (원인 → 결과)
    print(f"\n📋 인과 관계 상세:")
    print(f"  {'='*76}")
    
    has_relationships = False
    for var in res['order_names']:
        parents_list = res['parents_names'][var]
        if parents_list:
            has_relationships = True
            print(f"\n  🎯 결과 변수: {var}")
            print(f"     원인 변수들:")
            for parent in parents_list:
                print(f"       • {parent} → {var}")
    
    if not has_relationships:
        print("  (직접적인 인과 관계 발견되지 않음)")
    
    print(f"\n  {'='*76}")

print(f"\n{'='*80}")
print('✅ 확장 변수셋 BOSS 분석 완료')
print(f"{'='*80}")

확장된 변수셋 BOSS 인과 분석


Cluster 0 분석

📊 기본 정보:
  - 표본 수: 8,629개
  - 변수 수: 21개
  - 발견된 인과 관계: 115개

🔄 인과 순서 (왼쪽 → 오른쪽):
  여성_50대_비중 → 남성_30대_비중 → 남성_50대_비중 → 남성_20대이하_비중 → 여성_40대_비중 → 동일상권_내매출순위비율 → 유니크고객수_구간 → 거주고객_비율 → 여성_20대이하_비중 → 재방문_고객비중 → 신규_고객비중 → 직장인고객_비율 → 남성_40대_비중 → 유동인구고객_비율 → 운영개월수_구간 → 남성_60대이상_비중 → 여성_30대_비중 → 여성_60대이상_비중 → 객단가_구간 → 취소율_구간 → 배달매출_비율

🌟 원인 변수 (Root Causes) - 1개:
  ✓ 여성_50대_비중

📋 인과 관계 상세:

  🎯 결과 변수: 남성_30대_비중
     원인 변수들:
       • 여성_50대_비중 → 남성_30대_비중

  🎯 결과 변수: 남성_50대_비중
     원인 변수들:
       • 여성_50대_비중 → 남성_50대_비중

  🎯 결과 변수: 남성_20대이하_비중
     원인 변수들:
       • 여성_50대_비중 → 남성_20대이하_비중
       • 남성_30대_비중 → 남성_20대이하_비중

  🎯 결과 변수: 여성_40대_비중
     원인 변수들:
       • 남성_30대_비중 → 여성_40대_비중
       • 남성_50대_비중 → 여성_40대_비중

  🎯 결과 변수: 동일상권_내매출순위비율
     원인 변수들:
       • 남성_20대이하_비중 → 동일상권_내매출순위비율
       • 남성_30대_비중 → 동일상권_내매출순위비율
       • 남성_50대_비중 → 동일상권_내매출순위비율
       • 여성_50대_비중 → 동일상권_내매출순위비율
       • 여성_40대_비중 → 동일상권_내매출순위비율

  🎯 결과 변수: 유니크고객수_구간
     원인 변수들:
   

In [84]:
# 클러스터별 재방문_고객비중으로 향하는 모든 인과 경로 찾기 (2단계까지만)

def find_paths_limited(parents_dict, target, max_steps=2):
    """
    특정 변수(target)로 향하는 모든 인과 경로를 찾기 (최대 단계 제한)
    
    Parameters:
    -----------
    parents_dict: dict - 각 변수의 부모 노드 딕셔너리
    target: str - 목표 변수
    max_steps: int - 최대 단계 수 (화살표 개수)
    
    Returns:
    --------
    list of lists - 모든 경로들의 리스트
    """
    all_paths = []
    
    if target not in parents_dict or not parents_dict[target]:
        # 부모가 없으면 빈 리스트 반환
        return []
    
    # 1단계: 직접 부모 → target
    direct_parents = parents_dict[target]
    for parent in direct_parents:
        all_paths.append([parent, target])
    
    # 2단계 이상: 재귀적으로 탐색
    if max_steps >= 2:
        for parent in direct_parents:
            if parent in parents_dict and parents_dict[parent]:
                # parent의 부모들을 찾아서 경로 확장
                grandparent_paths = find_paths_limited(parents_dict, parent, max_steps - 1)
                for path in grandparent_paths:
                    # 경로 길이 확인 (화살표 개수 = 노드 개수 - 1)
                    if len(path) <= max_steps:
                        all_paths.append(path + [target])
    
    return all_paths


print('=' * 80)
print('클러스터별 재방문_고객비중으로 향하는 모든 인과 경로 (2단계까지)')
print('=' * 80)

target_var = '재방문_고객비중'

for cl in clusters:
    if cl not in boss_extended_results:
        continue
    
    res = boss_extended_results[cl]
    parents_dict = res['parents_names']
    
    print(f"\n{'='*80}")
    print(f"Cluster {cl}")
    print(f"{'='*80}")
    
    if target_var not in parents_dict:
        print(f"⚠️ {target_var}이(가) 분석 변수에 포함되지 않았습니다.")
        continue
    
    # 경로 찾기 (최대 2단계)
    all_paths = find_paths_limited(parents_dict, target_var, max_steps=2)
    
    if not all_paths:
        print(f"📭 {target_var}로 향하는 인과 경로가 없습니다. (원인 변수)")
    else:
        print(f"📊 총 {len(all_paths)}개의 인과 경로 발견 (2단계까지)\n")
        
        # 직접 경로와 간접 경로 구분
        direct_paths = [p for p in all_paths if len(p) == 2]  # 1단계 (노드 2개 = 화살표 1개)
        indirect_2step_paths = [p for p in all_paths if len(p) == 3]  # 2단계 (노드 3개 = 화살표 2개)
        
        # 직접 경로 (1단계)
        if direct_paths:
            print(f"🎯 1단계 직접 경로 ({len(direct_paths)}개):")
            for i, path in enumerate(direct_paths, 1):
                print(f"  {i}. {' → '.join(path)}")
        
        # 간접 경로 (2단계)
        if indirect_2step_paths:
            print(f"\n🔄 2단계 간접 경로 ({len(indirect_2step_paths)}개):")
            for i, path in enumerate(indirect_2step_paths, 1):
                print(f"  {i}. {' → '.join(path)}")

print(f"\n{'='*80}")
print('✅ 모든 클러스터 분석 완료 (2단계까지만)')
print(f"{'='*80}")

클러스터별 재방문_고객비중으로 향하는 모든 인과 경로 (2단계까지)

Cluster 0
📊 총 28개의 인과 경로 발견 (2단계까지)

🎯 1단계 직접 경로 (8개):
  1. 여성_20대이하_비중 → 재방문_고객비중
  2. 남성_50대_비중 → 재방문_고객비중
  3. 남성_20대이하_비중 → 재방문_고객비중
  4. 여성_40대_비중 → 재방문_고객비중
  5. 여성_50대_비중 → 재방문_고객비중
  6. 거주고객_비율 → 재방문_고객비중
  7. 유니크고객수_구간 → 재방문_고객비중
  8. 동일상권_내매출순위비율 → 재방문_고객비중

🔄 2단계 간접 경로 (20개):
  1. 남성_50대_비중 → 여성_20대이하_비중 → 재방문_고객비중
  2. 유니크고객수_구간 → 여성_20대이하_비중 → 재방문_고객비중
  3. 남성_20대이하_비중 → 여성_20대이하_비중 → 재방문_고객비중
  4. 여성_50대_비중 → 남성_50대_비중 → 재방문_고객비중
  5. 여성_50대_비중 → 남성_20대이하_비중 → 재방문_고객비중
  6. 남성_30대_비중 → 남성_20대이하_비중 → 재방문_고객비중
  7. 남성_30대_비중 → 여성_40대_비중 → 재방문_고객비중
  8. 남성_50대_비중 → 여성_40대_비중 → 재방문_고객비중
  9. 동일상권_내매출순위비율 → 거주고객_비율 → 재방문_고객비중
  10. 유니크고객수_구간 → 거주고객_비율 → 재방문_고객비중
  11. 남성_20대이하_비중 → 유니크고객수_구간 → 재방문_고객비중
  12. 남성_30대_비중 → 유니크고객수_구간 → 재방문_고객비중
  13. 여성_50대_비중 → 유니크고객수_구간 → 재방문_고객비중
  14. 여성_40대_비중 → 유니크고객수_구간 → 재방문_고객비중
  15. 동일상권_내매출순위비율 → 유니크고객수_구간 → 재방문_고객비중
  16. 남성_20대이하_비중 → 동일상권_내매출순위비율 → 재방문_고객비중
  17. 남성_30대_비중 → 동일상권_내매출순위비율 → 재방문_